In [226]:
import pandas as pd
import re
df = pd.read_csv('data.csv')
print(df)
title_norm = df['title'].str.upper()
print(title_norm)
#IMPORTACAO DO PANDAS E DAS LISTAS PYTHON

          date                                              title  amount
0   2025-08-30                                      Divino Eterno    5.00
1   2025-08-30                             Uber Uber *Trip Help.U   15.58
2   2025-08-29                             Uber Uber *Trip Help.U    5.92
3   2025-08-28                                     Joaopauloelias   12.50
4   2025-08-27                               Amazon Prime Aluguel    6.90
5   2025-08-26                             Uber Uber *Trip Help.U    9.02
6   2025-08-24                               Amazon Prime Aluguel    4.90
7   2025-08-24                       Estorno de "Burger King App"  -75.80
8   2025-08-24                                    Burger King App   75.80
9   2025-08-23                              Ifd*56.119.929 Yasmin   17.00
10  2025-08-23                             Uber Uber *Trip Help.U    6.87
11  2025-08-23                                         Uber* Trip    6.22
12  2025-08-23                        

LISTA DE CENTROS DE CUSTO

In [225]:
df_cf = pd.read_csv('categorias_financeiras_organizado.csv')
print(df_cf)

    CATEGORIA       TITULO
0    DELIVERY        IFOOD
1    DELIVERY        RAPPI
2    DELIVERY  ZE DELIVERY
3    DELIVERY    UBER EATS
4    DELIVERY      AIQFOME
..        ...          ...
138    CONTAS      ALUGUEL
139    CONTAS         IPTU
140    CONTAS         IPVA
141    CONTAS         DARF
142    CONTAS          DAS

[143 rows x 2 columns]


CRIACAO DO PRIMEIRO MODULO (ADICAO DE FILTROS - INICIALMENTE PRE DEFINIDOS - E ESTIPULACAO DE METAS PARA GASTOS)

In [ ]:
def adicionar_categoria_gastos():
    nome = input('Digite o nome do centro de custo desejado')
    nome_padrao = nome.upper().lstrip().rstrip()
    limite = float(input(f'Valor limite para gastos'))
    user = []
    save = 'BOTAO SALVAR'
    cancel = 'BOTAO CANCELAR'

    if df_cf['CATEGORIA'].isin([nome_padrao]).any():
        print('Essa categoria ja existe!')  
    else:
        novo_cc = {nome_padrao:limite}
        user.append(novo_cc)
        print(user)

adicionar_categoria_gastos()


In [ ]:
def botao_categorias():
    criar = 'CRIAR CENTRO DE CUSTO'
    centro_padrao = 'CENTROS DE CUSTOS PADROES'

    if criar == True:
        adicionar_categoria_gastos()
    elif centro_padrao == True:
        print('CENTROS DE CUSTOS PADROES')

botao_categorias()